# Исследование рынка недвижимости

1. Подготовить SQL-запросы и написать выводы для:
- Время активности объявлений;
- Сезонность объявлений;
- Анализ рынка недвижимости Ленобласти.

2. Создать визуализации в DataLens:
- Создать дашборд для отображения результатов.

### Ссылка на дашборд

Дашборд доступен по ссылке https://datalens.yandex/5zzb1mhyhsxqr

### Фильтрация данных перед созданием запросов

In [ ]:
-- Определим аномальные значения (выбросы) по значению перцентилей:
WITH limits AS (
    SELECT
        PERCENTILE_DISC(0.99) WITHIN GROUP (ORDER BY total_area) AS total_area_limit,
        PERCENTILE_DISC(0.99) WITHIN GROUP (ORDER BY rooms) AS rooms_limit,
        PERCENTILE_DISC(0.99) WITHIN GROUP (ORDER BY balcony) AS balcony_limit,
        PERCENTILE_DISC(0.99) WITHIN GROUP (ORDER BY ceiling_height) AS ceiling_height_limit_h,
        PERCENTILE_DISC(0.01) WITHIN GROUP (ORDER BY ceiling_height) AS ceiling_height_limit_l
    FROM real_estate.flats
),
-- Найдем id объявлений, которые не содержат выбросы:
filtered_id AS(
    SELECT id
    FROM real_estate.flats  
    WHERE 
        total_area < (SELECT total_area_limit FROM limits)
        AND (rooms < (SELECT rooms_limit FROM limits) OR rooms IS NULL)
        AND (balcony < (SELECT balcony_limit FROM limits) OR balcony IS NULL)
        AND ((ceiling_height < (SELECT ceiling_height_limit_h FROM limits)
            AND ceiling_height > (SELECT ceiling_height_limit_l FROM limits)) OR ceiling_height IS NULL)
    )
-- Выведем объявления без выбросов:
SELECT *
FROM real_estate.flats
WHERE id IN (SELECT * FROM filtered_id);

### 1. Время активности объявлений

Результат запроса должен ответить на такие вопросы:
1. Какие сегменты рынка недвижимости Санкт-Петербурга и городов Ленинградской области имеют наиболее короткие или длинные сроки активности объявлений?
2. Какие характеристики недвижимости, включая площадь недвижимости, среднюю стоимость квадратного метра, количество комнат и балконов и другие параметры, влияют на время активности объявлений? Как эти зависимости варьируют между регионами?
3. Есть ли различия между недвижимостью Санкт-Петербурга и Ленинградской области по полученным результатам?

In [ ]:
-- Подготовка данных: категоризация по СПб и ЛенОбл, категоризация по периодам активности. 
-- считаем стоимость кв.метра, фильтрация по типу "город"
WITH data_analysis AS (
	SELECT 
		CASE 
			WHEN city = 'Санкт-Петербург' 
				THEN  'Санкт-Петербург'
			ELSE 'Ленинградская область'
		END AS region_city,
		CASE 
			WHEN days_exposition <= 30 
				THEN  'До месяца'
			WHEN days_exposition <= 90
				THEN 'До квартала'
			WHEN days_exposition <= 180
				THEN 'До полугода'
			WHEN days_exposition > 180
				THEN 'Более полугода'
			ELSE 'Объявление активно'
		END AS period_exposition,

		round(avg((last_price/total_area)::numeric)) AS avg_price_total_area, -- Средняя стоимость кв. метра
		count(id) AS cnt_advertisement, -- Кол-во объявлений

		PERCENTILE_CONT(0.50) WITHIN GROUP (ORDER BY total_area) AS median_total_area, -- Медиана по общей площади
		round(avg(total_area::numeric), 1) AS avg_total_area, -- средняя общей площади

		PERCENTILE_CONT(0.50) WITHIN GROUP (ORDER BY rooms) AS median_cnt_room, -- Медиана по кол-ву комнат
		round(avg(rooms::numeric), 1) AS avg_rooms, -- средняя по комнатам

		PERCENTILE_CONT(0.50) WITHIN GROUP (ORDER BY balcony) AS median_cnt_balcony, --  Медиана по кол-ву балконов
		round(avg(balcony::numeric), 1) AS avg_balcony, -- средняя по балконам

		PERCENTILE_CONT(0.50) WITHIN GROUP (ORDER BY floor) AS median_floor, -- Медиана по этажу
		round(avg(floor)::numeric, 1) AS avg_floor, -- средняя по этажу

		round(avg(ceiling_height::numeric), 1) AS avg_ceiling_height, -- средняя высота потолка
		round(avg(airports_nearest::numeric), 1) AS avg_airports_nearest, -- среднее расстояние до ближ. аэропорта

		PERCENTILE_CONT(0.50) WITHIN GROUP (ORDER BY parks_around3000) AS median_parks_around3000, -- медиана по числу парков в радиусе 3км
		PERCENTILE_CONT(0.50) WITHIN GROUP (ORDER BY ponds_around3000) AS median_ponds_around3000, -- медиана по числу водоемов в радиусе 3км

		count(id) FILTER(WHERE is_apartment = 1) AS cnt_apartment, -- кол-во апартаментов
		count(id) FILTER(WHERE open_plan = 1) AS cnt_open_plan -- кол-во с открытой планировкой

	FROM 
        real_estate.advertisement

	LEFT JOIN real_estate.flats USING(id)
	LEFT JOIN real_estate.city USING(city_id)
	LEFT JOIN real_estate.type USING(type_id)

	WHERE 
        id IN (SELECT * FROM filtered_id) 
        AND type = 'город' AND EXTRACT(year FROM first_day_exposition) != 2014 
        AND EXTRACT(year FROM first_day_exposition) != 2019
	GROUP BY 
        region_city, period_exposition
)

SELECT 
    *,
	round(cnt_advertisement*100/sum(cnt_advertisement) OVER(PARTITION BY region_city), 1) AS perc_adv_region, -- доля по региону
	round(cnt_apartment*100/sum(cnt_advertisement) OVER(PARTITION BY region_city), 2) AS perc_apart, -- доля апартаментов
	round(cnt_open_plan*100/sum(cnt_advertisement) OVER(PARTITION BY region_city), 2) AS perc_open_plan -- доля с открытой планировкой
FROM data_analysis;

[![Ex1.png](https://i.postimg.cc/C1rXNXR8/Ex1.png)](https://postimg.cc/sQSTfn9f)

### Вывод:

1. Какие сегменты рынка недвижимости Санкт-Петербурга и городов Ленинградской области имеют наиболее короткие или длинные сроки активности объявлений?

В Санкт-Петербурге наиболее короткие сроки (до месяца) у квартир на 5 этаже со средней стоимость кв.м. 108 920 руб (самая низкая цена в разрезе по срокам активности), со средней площадью 49 кв м., с 2 комнатами  и 1 балконом, высотой потолков 2,8 м. Такие объявления составляют 16% от всех по Санкт-Петербургу.
Длительные сроки (более полугода) у квартир на 5 этаже, со средней стоимостью кв.м. 114 981 руб (самая высокая цена в разрезе сроков, без учета непроданных), со ср. площадью 65,8 кв.м., с 2 комнатами и 1 балконом, высотой потолков 2,8 м. Такие объявления составляют 31% по Санкт-Петербургу.
В Ленинградской области, наиболее короткие сроки у квартир на 4 этаже, со средней стоимостью кв.м. 71 908 руб (самая высокая цена в разрезе сроков), со средней площадью 48,8 кв.м., с 2 комнатами и 1 балконом, высотой потолков 2,7 м. Такие объявления составляют 12% по Санкт-Петербургу.
Длительные сроки у сегмента квартир на 3 этаже, средней стоимость кв.м. 68 215 руб, со средней площадью 55 кв.м., с 2 комнатами и 1 балконом, высотой потолков 2,7 м. Такие объявления составляют 31% по Санкт-Петербургу.

2. Какие характеристики недвижимости, включая площадь недвижимости, среднюю стоимость квадратного метра, количество комнат и балконов и другие параметры, влияют на время активности объявлений? Как эти зависимости варьируют между регионами?

На время активности объявлений влияют следующие характеристики:
- Стоимость кв.метра - квартиры с высокой стоимость кв.м. Продаются дольше. При этом, такая зависимость характерна только для Санкт-Петербурга, в ЛенОбл. такой зависимости нет;
- Кол-во кв. метров - чем больше квадратных метров, тем дольше продается квартира. Тенденция характерна и для СПб и для ЛО;
- Кол-во комнат - трудно рассмотреть зависимость, т.к. медиана показывает, что во всех сегментах квартиры двухкомнатные. Если ориентироваться на среднее значение, то еле просматривается прямая зависимость кол-во комнат и времени объявления;
- Аналогично с кол-ом балконов; 
- Этаж не влияет на длительность объявления.

3. Есть ли различия между недвижимостью Санкт-Петербурга и Ленинградской области по полученным результатам?
Из полученных результатов выявлены следующие различия:

- Средняя стоимость кв.м. в Санкт-Петербурге выше, чем в ЛенОбласти, при этом только в СПб это влияет на длительность объявления;
- В ЛО продаются квартиры меньшей площадью, чем в СПб, при этом у них одинаковая зависимость между длительностью объявления и кол-вом кв.м.;
- В СПб и ЛО в среднем одинаковое кол-во комнат и балконов, при этом в СПб продаются квартиры на более высоком этаже, чем в ЛО.

---

### 2. Сезонность объявлений

Результат запроса должен ответить на такие вопросы:
1. В какие месяцы наблюдается наибольшая активность в публикации объявлений о продаже недвижимости? А в какие — по снятию? Это показывает динамику активности покупателей.
2. Совпадают ли периоды активной публикации объявлений и периоды, когда происходит повышенная продажа недвижимости (по месяцам снятия объявлений)?
3. Как сезонные колебания влияют на среднюю стоимость квадратного метра и среднюю площадь квартир? Что можно сказать о зависимости этих параметров от месяца?

In [ ]:
WITH table_first AS (  -- таблица с выставленными объявлениями
	SELECT 
		EXTRACT(month FROM first_day_exposition) AS month, -- месяц
		count(a.id) AS cnt_first_exp, -- кол-во объявлений
		round(avg(last_price/total_area)::numeric, 2) AS avg_price_total_area_first_exp, -- Средняя стоимость кв. метра
		round(avg(total_area)::numeric, 2) AS avg_total_area_first_exp -- Средняя кол-во кв. метров

	FROM real_estate.advertisement AS a
	LEFT JOIN real_estate.flats USING(id)
	LEFT JOIN real_estate.type USING(type_id)

	WHERE id IN (SELECT * FROM filtered_id) 
		AND EXTRACT(year FROM first_day_exposition) != 2014 AND EXTRACT (year FROM first_day_exposition) != 2019 
		AND type = 'город'

	GROUP BY month 
	ORDER BY month
),

table_last AS ( -- таблица с завершенными объявлениями
	SELECT 
		EXTRACT(month FROM first_day_exposition + days_exposition::int) AS month, -- месяц
		count(a.id) AS cnt_last_exp, -- кол-во завершенных объявлений
		round(avg(last_price/total_area)::numeric, 2) AS avg_price_total_area_last_exp, -- Средняя стоимость кв. метра
		round(avg(total_area)::numeric, 2) AS avg_total_area_last_exp -- Средняя кол-во кв. метров

	FROM real_estate.advertisement AS a
	LEFT JOIN real_estate.flats USING(id)
	LEFT JOIN real_estate.type USING(type_id)

	WHERE id IN (SELECT * FROM filtered_id) 
		AND EXTRACT(year FROM first_day_exposition) != 2014 AND EXTRACT (year FROM first_day_exposition) != 2019 
		AND (first_day_exposition + days_exposition::int) IS NOT NULL 
		AND type = 'город'

	GROUP BY month
	ORDER BY month
)

SELECT 
	month, 															-- месяц
	cnt_first_exp, 													-- кол-во опубликованных объявлений
	rank() OVER(ORDER BY cnt_first_exp desc) AS rank_first_exp, 	-- ранг опубликованных
	cnt_last_exp, 													-- кол-во завершенных объявлений
	rank() OVER(ORDER BY cnt_last_exp desc) AS rank_last_exp, 		-- ранг завершенных
	round(cnt_last_exp*100/cnt_first_exp::numeric, 2) AS perc_last_exp, -- доля завершенных
	avg_price_total_area_first_exp, 								-- Средняя стоимость кв. метра у опубликованных
	avg_price_total_area_last_exp, 									-- Средняя стоимость кв. метра у завершенных
	avg_total_area_first_exp,										-- Средняя кол-во кв. метров у опубликованных
	avg_total_area_last_exp 										-- Средняя кол-во кв. метров у завершенных

FROM table_first AS f
FULL JOIN table_last AS l USING(month)

ORDER BY month;

In [ ]:
-- Второй вариант выполнения

WITH table_first AS (
	SELECT 
		EXTRACT(month FROM first_day_exposition) AS month,
		EXTRACT(year FROM first_day_exposition) AS year,
		count(a.id) AS cnt_first_exp,
		round(avg(last_price/total_area)::numeric, 2) AS avg_price_total_area_first_exp, -- Средняя стоимость кв. метра
		round(avg(total_area)::numeric, 2) AS avg_total_area_first_exp -- Средняя кол-во кв. метров

	FROM real_estate.advertisement AS a
	LEFT JOIN real_estate.flats USING(id)

	WHERE 
		id IN (SELECT * FROM filtered_id) 
		AND EXTRACT(year FROM first_day_exposition) != 2014 
		AND EXTRACT(year FROM first_day_exposition) != 2019

	GROUP BY month, year 
	ORDER BY month
),

table_last AS (
	SELECT 
		EXTRACT(month FROM first_day_exposition + days_exposition::int) AS month,
		EXTRACT(year FROM first_day_exposition + days_exposition::int) AS year,
		count(a.id) AS cnt_last_exp,
		round(avg(last_price/total_area)::numeric, 2) AS avg_price_total_area_last_exp, -- Средняя стоимость кв. метра
		round(avg(total_area)::numeric, 2) AS avg_total_area_last_exp -- Средняя кол-во кв. метров

	FROM real_estate.advertisement a
	LEFT JOIN real_estate.flats USING(id)

	WHERE id IN (SELECT * FROM filtered_id) 
		AND EXTRACT(year FROM first_day_exposition) != 2014
		AND EXTRACT (year FROM first_day_exposition) != 2019 
		AND (first_day_exposition + days_exposition::int) IS NOT NULL

	GROUP BY month, year
	ORDER BY month
)

SELECT 
	f.year,
	f.month,
	cnt_first_exp,
	rank() OVER(ORDER BY cnt_first_exp desc) AS rank_first_exp,
	cnt_last_exp,
	rank() OVER(ORDER BY cnt_last_exp desc) AS rank_first_exp,
	round(cnt_last_exp*100/cnt_first_exp::numeric, 2) AS perc_last_exp, -- доля завершенных
	avg_price_total_area_first_exp,
	avg_price_total_area_last_exp,
	avg_total_area_first_exp,
	avg_total_area_last_exp

FROM table_first AS f
FULL JOIN table_last AS l ON f.month = l.month AND f.year = l.year

WHERE 
	l.year IS NOT NULL 
	AND f.year IS NOT NULL
ORDER BY year, month;

[![Ex2.png](https://i.postimg.cc/xTfgCtqD/Ex2.png)](https://postimg.cc/MXL130yd)

### Вывод:

1. В какие месяцы наблюдается наибольшая активность в публикации объявлений о продаже недвижимости? А в какие — по снятию? Это показывает динамику активности покупателей. Таблица обновлена
    
    Топ-4 по публикации (месяц, кол-во)
    1. Ноябрь - 1569
    2. Октябрь - 1437
    3. Январь - 1369
    4. Сентябрь - 1341	

    Топ-4 по снятию (месяц, кол-во)
    1. Октябрь - 1360
    2. Ноябрь - 1301
    3. Сентябрь - 1238
    4. Январь - 1225


2. Совпадают ли периоды активной публикации объявлений и периоды, когда происходит повышенная продажа недвижимости (по месяцам снятия объявлений)?

Периоды совпадают с разницей в 1 месяц, т.е. покупка и продажа недвижимости происходит в соседней паре месяцев (или квартале). Это может свидетельствовать о том, что сначала покупают новую квартиру, а потом продают старую. Преобладают объявления осенью, причем публикация и продажа совпадают.

3. Как сезонные колебания влияют на среднюю стоимость квадратного метра и среднюю площадь квартир? Что можно сказать о зависимости этих параметров от месяца?

В летние месяцы средняя стоимость продажи и покупки квартир ниже, чем в осенне-зимние, а также кол-во объявлений меньше. Это может свидетельствовать о том, что летом люди меньше покупают из-за периода отпусков, при этом в осенние и зимние месяцы больше продают квартир. 


---

### 3. Анализ рынка недвижимости Ленобласти

Результат запроса должен ответить на такие вопросы:
1. В каких населённые пунктах Ленинградской области наиболее активно публикуют объявления о продаже недвижимости?
2. В каких населённых пунктах Ленинградской области — самая высокая доля снятых с публикации объявлений? Это может указывать на высокую долю продажи недвижимости.
3. Какова средняя стоимость одного квадратного метра и средняя площадь продаваемых квартир в различных населённых пунктах? Есть ли вариация значений по этим метрикам?
4. Среди выделенных населённых пунктов какие пункты выделяются по продолжительности публикации объявлений? То есть где недвижимость продаётся быстрее, а где — медленнее.

In [ ]:
-- Подготовка данных: категоризация по СПб и ЛенОбл, категор. по перодам активности. + считаем стоим. кв.метра, фильтр по типу "город"
WITH len_obl_filter AS (
	SELECT 
		count(id) AS cnt_id

	FROM real_estate.advertisement
	FULL JOIN real_estate.flats USING(id)
	FULL JOIN real_estate.city USING(city_id)
	
	WHERE 
	id IN (SELECT * FROM filtered_id) 
	AND city != 'Санкт-Петербург'
),

data_analysis AS (
	SELECT
		NTILE(4) OVER(ORDER BY avg(days_exposition)) AS rank,
		city,

		-- Кол-во объявлений и доля от всех объявлений
		count(id) AS cnt_advertisement, 
		round(count(id)::numeric/(SELECT cnt_id	FROM len_obl_filter)*100, 2) AS perc_cnt_adv,

		-- кол-во завершенных объявлений и доля завершенных объявлений от всех н/п
		count(days_exposition) AS cnt_exp, 
		round(count(days_exposition)*100/count(id)::numeric, 2) AS perc_cnt_exp,

		-- Медиана по длительности продажи и средняя по длительности продажи
		PERCENTILE_CONT(0.50) WITHIN GROUP (ORDER BY days_exposition) AS median_cnt_days_exp,
		round(avg(days_exposition)::numeric, 2) AS avg_days_exp, 

		-- средняя общая площадь и Средняя стоимость кв. метра
		round(avg(total_area)::numeric, 2) AS avg_total_area,
		round(avg(last_price/total_area)::numeric) AS avg_price_total_area,

		-- Медиана по кол-ву комнат и средняя по комнатам
		PERCENTILE_CONT(0.50) WITHIN GROUP (ORDER BY rooms) AS median_cnt_room, 
		round(avg(rooms)::numeric, 2) AS avg_rooms,
		 
		-- Медиана по кол-ву балконов и средняя по балконам
		PERCENTILE_CONT(0.50) WITHIN GROUP (ORDER BY balcony) AS median_cnt_balcony, 
		round(avg(balcony)::numeric, 2) AS avg_balcony,

		-- медиана по этажу и средняя по этажу
		PERCENTILE_CONT(0.50) WITHIN GROUP (ORDER BY floor) AS median_floor,
		round(avg(floor)::numeric, 2) AS avg_floor

	FROM real_estate.advertisement
	LEFT JOIN real_estate.flats USING(id)
	LEFT JOIN real_estate.city USING(city_id)
	
	WHERE 
		id IN (SELECT * FROM filtered_id) 
		AND city != 'Санкт-Петербург' 
		AND EXTRACT(year FROM first_day_exposition) != 2014 
		AND EXTRACT(year FROM first_day_exposition) != 2019
	
	GROUP BY city
	ORDER BY cnt_advertisement DESC
)

SELECT *
FROM data_analysis

ORDER BY 
	cnt_advertisement DESC, 
	perc_cnt_exp DESC
LIMIT 15; -- Выбраны города с наибольшим кол-вом объявлений и наибольшей долей. 
-- Объявлений со 100% долей продаж крайне мало, по такому кол-ву данных нельзя делать выводы

[![Ex3.png](https://i.postimg.cc/L5wsrxbY/Ex3.png)](https://postimg.cc/bS08n98p)

### Вывод:

1. В каких населённые пунктах Ленинградской области наиболее активно публикуют объявления о продаже недвижимости?

    Топ-5 н/п по кол-ву объявлений в ЛенОбласти:
    1.	Мурино - 519 (или 8,2% от всех объявлений)
    2.	Кудрово - 429 (или 6,8%)
    3.	Шушары - 375 (или 5,9%)
    4.	Всеволожск - 324 (или 5,1%)
    5.	Парголово - 285 (или 4,5%)

2. В каких населённых пунктах Ленинградской области — самая высокая доля снятых с публикации объявлений? Это может указывать на высокую долю продажи недвижимости.

    Топ-5 н/п доли снятых объявлений (кол-во + доля от объявлений в этом же н/п):
    6.	Бугры - 87 (или 98,9% от объявлений в этом н/п)
    7.	Мурино - 512 (или 98,7%)
    8.	Парголово - 280 (или 98.3%)
    9.	Кудрово - 421 (или 98,1%)
    10.	Красное Село - 118 (или 97,5%)

3. Какова средняя стоимость одного квадратного метра и средняя площадь продаваемых квартир в различных населённых пунктах? Есть ли вариация значений по этим метрикам?

- Средняя площадь варьируется от 43,8 до 61,4 кв.м. Есть предположение, что быстрее всего продаются квартиры с ср.площадью до 50 кв.м
- Средняя стоимость кв.м. варьируется от 57 620 руб до 104 339 руб. Зависимости не обнаружил, т.к. Квартиры, например, в  Буграх (80 369 руб) и Мурино (85 630 руб) продаются активнее, чем в Выборге за 57 620 руб.

4. Среди выделенных населённых пунктов какие пункты выделяются по продолжительности публикации объявлений? То есть где недвижимость продаётся быстрее, а где — медленнее.
- Медиана и среднее по продолжительности публикации сильно отличаются, что говорит о выбросах в сторону больших значений. 
- Если ориентироваться на средние значения, то быстрее всего продается в н/п Колпино (140 дней) и Мурино (150 дней), дольше - в Красное Село (211 дней) и Сестрорецк (207 дней)
- Медиана показывает, что быстрее всего продаются в н/п Кудрово (75 дней) и Мурино (78 дня), дольше - в Пушкин (130 дней) и Красное Село (142 дней)

---

### Общий вывод и рекомендации

В ходе анализа выявлены следующие инсайты и тенденции:
1.	При планировании бизнес-стратегии стоит ориентироваться на более ликвидные сегменты рынка недвижимости. Согласно выгрузке данных, такими сегментами являются:
    - В Санкт-Петербурге квартиры на 5 этаже, со средней площадью 49  кв м., с 2 комнатами  и 1 балконом и стоимостью 108 920 руб за кв.м;
    - В Ленинградской области квартиры на 4 этаж,  средней площадью 48,8  кв.м., с 2 комнатами и 1 балконом и стоимостью 71 908 руб за кв.м.;
2.	В Санкт-Петербурге и Ленинградской области существуют сезонные тенденции. Больше объявлений и продаж квартир происходят в осенний (и зимний) период. Меньше - в летний период. При этом, стоит заметить, что в летние периоды стоимость квартир дешевеет, а в осенне-зимние наоборот, цены растут;
3.	Наиболее привлекательные населенные пункты в Ленинградской области по кол-ву объявлений и самой высокой доле продаж: Мурино, Кудрово, Парголово. При этом, рекомендую дополнительно приглядеться к Колпино и Бугры, т.к. в этих н/п быстрее всего продается недвижимость - возможно, это привлекательные, перспективные районы ЛО.

Рекомендации:
1.	Для более эффективной бизнес-стратегии необходимо посмотреть более перспективные и ликвидные районы Санкт-Петербурга, узнать о запланированных строительствах новостройках;
2.	Провести дополнительный анализ продаж н/п в Ленинградской области на предмет перспективных и привлекательных районов.
